In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({'id':'174Ws0LnaEQdK7UmOKh-8GJwftO18Mfej'})
your_module.GetContentFile('metric.py')
your_module1 = drive.CreateFile({'id':'11ssSCDodnZ6_jXcHLZ8AqYHbg7g7K8Be'})
your_module1.GetContentFile('threshold.py')
your_module2 = drive.CreateFile({'id':'1JpuTOGEKMq9iZcO6Tp4ET9icd5EG0P0D'})
your_module2.GetContentFile('cat_list.py')
your_module2 = drive.CreateFile({'id':'1gsMrGL0BfVASQICf-eesMRelmCgMLTYs'})
your_module2.GetContentFile('cat_list_show.py')
from metric import *
from threshold import *
from cat_list import *
from cat_list_show import *

In [ ]:
def jacard(y_test, predictions):
    accuracy = 0.0

    for i in range(y_test.shape[0]):
        intersection = 0.0
        union = 0.0
        for j in range(y_test.shape[1]):
            if int(y_test[i,j]) == 1 or int(predictions[i,j]) == 1:
                union += 1
            if int(y_test[i,j]) == 1 and int(predictions[i,j]) == 1:
                intersection += 1
            if int(y_test[i,j]) == 2 or int(predictions[i,j]) == 2:
                union += 1
            if int(y_test[i,j]) == 2 and int(predictions[i,j]) == 2:
                intersection += 1
            
        if union != 0:
            accuracy = accuracy + float(intersection/union)

    accuracy = float(accuracy/y_test.shape[0])

    return accuracy

In [ ]:
!pip install tensorflow-determinism

In [ ]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(sd)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_floatx('float64')
initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)
#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1
       ,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)

x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))
print (subsetAccuracy(y_test, p))

In [ ]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(sd)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_floatx('float64')
initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)
#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=LSTM(units=256, return_sequences=True,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)
x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))
print (subsetAccuracy(y_test, p))

In [ ]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(sd)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_floatx('float64')
initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)
#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=GRU(units=256, return_sequences=True,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)
x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))
print (subsetAccuracy(y_test, p))

In [ ]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(sd)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_floatx('float64')
initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D,Bidirectional
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)
#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=Bidirectional(LSTM(units=256, return_sequences=True,kernel_initializer=initializer,use_bias=False))(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)
x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))
print (subsetAccuracy(y_test, p))

In [ ]:
!pip install attention
from attention import Attention

In [ ]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(sd)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_floatx('float64')
initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D,Bidirectional,Flatten
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization,MaxPooling1D
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)

#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x = Conv1D(256, 3, activation='relu')(x) ## conv layer 128 filters with a size of 2
x = MaxPooling1D(1)(x)
x = Attention()(x)
x = Flatten()(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))
print (subsetAccuracy(y_test, p))

In [ ]:
from sklearn.metrics import classification_report
for i in range(9):
  print(category_list[i])
  print(classification_report(y_test[:,i], p[:,i]))